In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.notebook import tqdm
import re 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service


In [2]:
# 서울시 숙소 리스트 로드
df = pd.read_csv('../DAIN/crw_list.txt')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 610 entries, 0 to 609
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  610 non-null    int64  
 1   사업장명        610 non-null    object 
 2   지번주소        610 non-null    object 
 3   도로명주소       602 non-null    object 
 4   위도          583 non-null    float64
 5   경도          583 non-null    float64
 6   인허가일자       610 non-null    object 
 7   인허가취소일자     1 non-null      float64
 8   영업상태코드      610 non-null    int64  
 9   폐업일자        136 non-null    object 
 10  휴업시작일자      9 non-null      object 
 11  휴업종료일자      9 non-null      object 
 12  재개업일자       0 non-null      float64
dtypes: float64(4), int64(2), object(7)
memory usage: 62.1+ KB


In [10]:
# 크롬 드라이버 시작
chrome_options = webdriver.ChromeOptions()              # 옵션값 사용
chrome_options.add_experimental_option("detach",True)   # 브라우저 꺼짐 방지
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install(), options = chrome_options))
#driver = webdriver.Chrome()

In [11]:
driver.get("https://map.naver.com/v5")
driver.implicitly_wait(5)
#driver.maximize_window()

In [12]:
# 검색 트리
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('이비스 앰배서더 명동', ',  ', '서울특별시 중구 명동1가 59-5')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('호텔 데님(Hotel Denim)',', ', '서울특별시 강남구 논현로 66 (개포동)')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('티마크그랜드호텔명동', ', ', '서울특별시 중구 회현동1가 194-15')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('위너스', ', ', '서울특별시 금천구 독산동 335-21')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('데이즈 호텔 명동', ', ', '서울특별시 중구 충무로1가 24-30')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('삼화', ', ', '서울특별시 관악구 신림동 1432-62')
#driver.find_element(By.CLASS_NAME, 'input_search').send_keys('나이스타임 관광호텔', ', ', '서울특별시 관악구 신림동 1422-18')
driver.find_element(By.CLASS_NAME, 'input_search').send_keys('노보텔 앰배서더', ', ', '서울 동대문 호텔, 서울특별시 중구 을지로6가 58-5')
driver.find_element(By.CLASS_NAME, 'input_search').send_keys(Keys.ENTER)
driver.implicitly_wait(5)
driver.switch_to.frame("searchIframe")

In [18]:
#식당 정보가 있다면 첫번째 식당의 url을 가져오기    
search_result = driver.find_elements(by=By.XPATH, value='//*[@id="_pcmap_list_scroll_container"]/ul/li')
count_search_result = len(driver.find_elements(by=By.XPATH, value='//*[@id="_pcmap_list_scroll_container"]/ul/li'))
print(f"주소검색결과 개수: {count_search_result}")
if count_search_result == 0:   
    pass
elif count_search_result > 1:
    #식당 정보 클릭        
    driver.execute_script('return document.querySelector("#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div> a:nth-child(1)").click()')
    driver.implicitly_wait(2)
else:
    print('검색 결과 한개, 새창 없음')

주소검색결과 개수: 4
[<selenium.webdriver.remote.webelement.WebElement (session="b593f31a93e97c1982b6a0de2f67f49e", element="9DC69E89A3790E26504264CF4486942C_element_409")>, <selenium.webdriver.remote.webelement.WebElement (session="b593f31a93e97c1982b6a0de2f67f49e", element="9DC69E89A3790E26504264CF4486942C_element_410")>]


In [9]:
# 검색후 오른쪽 확장 탭으로 전환
driver.switch_to.default_content()
detail_iframe = driver.find_element(By.ID, 'entryIframe')
driver.switch_to.frame(detail_iframe)
# driver.switch_to.frame('entryIframe') # 그냥하면 안됨


star_score = None
stn_type = None
visitor_review_count = None
blog_review_count = None
dist_from_stn = None
time_from_stn = None
facilitys_list_join = None
these_good_count = None
these_good_list_join = None


# 별점
try:
    star_score = float(driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div.place_section.OP4V8 > div.zD5Nm.f7aZ0 > div.dAsGb > span.PXMot.LXIwF > em').text)
except:
    pass
print("star_score: ", star_score)
# 업소유형
try:
    stn_type = driver.find_element(By.CSS_SELECTOR, '#_title > span.DJJvD').text
except:
    pass
print("stn_type: ", stn_type)
# 블로그 리뷰 수 & 방문자 리뷰 수
# 방문자
try:
    visitor_review_count = int(driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a/em').text)
except:
    pass
print("visitor_review_count: ", visitor_review_count)
# 블로그
try:
    blog_review_count = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[3]/a/em').text
except:
    pass
print("blog_review_count: ", blog_review_count)
# 가까운 지하철 역에서 거리 / 도보시간
# 거리
try:
    dist_from_stn = driver.find_element(By.CSS_SELECTOR, 'div.O8qbU.tQY7D > div > div > em').text
except:
    pass
print("dist_from_stn: ", dist_from_stn)
# 시간
try:
    time_text = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/div/div[2]/div/div/div/span').text
    #print("time_text: ", time_text)
    p = re.compile('[0-9]+분')
    time_from_stn = p.findall(time_text)            
except:
    pass
print("time_from_stn: ", time_from_stn)
# 주요시설
facilitys_list = []
try:
    facilitys = driver.find_elements(By.CSS_SELECTOR, 'div.place_section_content > div > div.fusPl > ul > li')
    for li in facilitys:
        facilitys_list.append(li.text.strip())
except:
    facilitys_list = None
#facilitys_list_join = "\n".join(facilitys_list)
facilitys_list_join = ",".join(facilitys_list)
print("facilitys_list: ", facilitys_list)

# .. 리뷰 클릭
try:    # 홈, 리뷰 탭이 있는지 없는지 확인
    check_list = driver.find_elements(By.CSS_SELECTOR, 'div.place_fixed_maintab > div > div > div > div.flicking-camera > a.tpj9w._tab-menu > span.veBoZ')
    for i, li in enumerate(check_list):
        print(f"print check_list {i}: ", li.text)
except:
    pass
else:
    # 리뷰 버튼이 있으면 클릭
    if check_list:
        for i, li in enumerate(check_list):
            #print(i, type(li.text), li.text)
            if "리뷰" in li.text: 
                find_i = i+1  # python index + 1
                driver.find_element(By.CSS_SELECTOR, f'div.place_fixed_maintab > div > div > div > div.flicking-camera > a:nth-child({find_i})').click()
                check_list = driver.find_elements(By.CSS_SELECTOR, 'div.place_fixed_maintab > div > div > div > div.flicking-camera > a > span.veBoZ')

                # 이런점이 좋아요 정보 크롤링
                these_good_list = []
                try:
                    these_good_count = int(driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div.place_section.no_margin.mdJ86 > div > div > div._Wmab > em').text)
                    print('these_good_count: ', these_good_count)
                except:
                    print("이런점이 좋아요 정보가 없습니다")
                    these_good_list = None                
                else:
                    # "이런점이 좋았어요" 더보기 다 눌러 놓기
                    while True:
                        try:
                            driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(7) > div:nth-child(3) > div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > a.Tvx37').click()
                            driver.implicitly_wait(1)
                        except:
                            break
                        
                    # 이런점이 좋아요, 딕셔너리로 {'리뷰':좋아요개수}
                    these_good = driver.find_elements(By.CSS_SELECTOR, 'div.place_section.no_margin.mdJ86 > div > div > div.k2tmh > ul > li')
                    for i, li in enumerate(these_good):
                        these_good_list.append({li.text.split("\n")[0].strip().replace('"',""): int(li.text.split("\n")[2].strip())})
                    print("these_good_list: ", these_good_list)
                    these_good_list_join=''
                    for i, item in enumerate(these_good_list):
                        #print(str(item))
                        if i == len(these_good_list)-1:
                            these_good_list_join += str(item)
                        else:
                            #these_good_list_join += str(item) + '\n'
                            these_good_list_join += str(item) + ','


star_score:  None
stn_type:  호스텔
visitor_review_count:  None
blog_review_count:  None
dist_from_stn:  220m
time_from_stn:  None
facilitys_list:  []
print check_list 0:  홈
print check_list 1:  사진


In [58]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="1d401a8854411add424658c3c27d516c")>>